In [6]:

# main.py 또는 api_service.py 등
import os
from dotenv import load_dotenv
import openai

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# os.environ을 사용하여 환경 변수를 가져옵니다.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
STABILITY_API_KEY = os.getenv("STABILITY_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되어 있지 않습니다.")

print(OPENAI_API_KEY[:10])
print(STABILITY_API_KEY[:10])

# 이제 'client' 객체를 사용하여 API를 호출할 수 있습니다.

sk-svcacct
sk-8hOadm1


In [3]:
import os
import requests
import base64
from PIL import Image # Pillow 라이브러리 import
import io # 이미지를 메모리 상에서 다루기 위해 import

def generate_edited_image(original_image_path: str, user_prompt: str):
    """
    주어진 이미지를 리사이즈하고, 원본 이미지를 기반으로 새로운 이미지를 생성합니다.
    """
    api_host = 'https://api.stability.ai'
    api_key = os.getenv("STABILITY_API_KEY")
    if not api_key:
        raise ValueError("STABILITY_API_KEY 환경 변수를 설정해주세요.")

    engine_id = "stable-diffusion-xl-1024-v1-0"

    # --- ✨ 이미지 리사이즈 로직 추가 ✨ ---
    # 1. 이미지 열기
    img = Image.open(original_image_path)
    
    # 2. SDXL이 허용하는 크기(1024x1024)로 리사이즈
    # Image.Resampling.LANCZOS는 고품질 리사이즈 필터입니다.
    img = img.resize((1024, 1024), Image.Resampling.LANCZOS)

    # 3. 리사이즈된 이미지를 메모리 버퍼에 저장 (임시 파일 생성 방지)
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    buffer.seek(0)
    # ------------------------------------

    # API에 보낼 데이터 구성
    form_data = {
        "image_strength": 0.5,
        "init_image_mode": "IMAGE_STRENGTH",
        "text_prompts[0][text]": user_prompt,
        "cfg_scale": 7,
        "samples": 1,
        "steps": 30,
    }

    # API 호출 시 파일 부분에 리사이즈된 이미지 버퍼를 전달
    response = requests.post(
        f"{api_host}/v1/generation/{engine_id}/image-to-image",
        headers={
            "Accept": "application/json",
            "Authorization": f"Bearer {api_key}"
        },
        files={
            'init_image': buffer  # 수정된 부분
        },
        data=form_data
    )

    if response.status_code != 200:
        raise Exception("API Error: " + response.text)

    data = response.json()

    output_path = "edited_image.png"
    for image in data["artifacts"]:
        with open(output_path, "wb") as f:
            f.write(base64.b64decode(image["base64"]))

    print(f"✅ 새로운 이미지가 '{output_path}'로 저장되었습니다.")
    return output_path

# --- 실행 ---
if __name__ == "__main__":
    my_image_file = "/Users/seoyeong/Desktop/앤해서웨이.jpeg"
    my_text_prompt = "Completely transform the original image's mood into a different one. If the original image is dark and mysterious, make it bright and whimsical. If it's modern and clean, make it ancient and rugged. Preserve only the main subject's silhouette or pose, but shift everything else — color scheme, background, lighting, and artistic style — into an entirely new mood or genre."

    generate_edited_image(my_image_file, my_text_prompt)

✅ 새로운 이미지가 'edited_image.png'로 저장되었습니다.


In [ ]:
import os
import openai

def generate_image_with_dalle3(prompt: str):
    """
    DALL-E 3를 사용하여 주어진 프롬프트로 이미지를 생성합니다.
    """
    try:
        # OpenAI 클라이언트 초기화 (환경 변수 사용 권장)
        client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        
        print("🎨 DALL-E 3 이미지 생성을 시작합니다...")
        print(f"   프롬프트: {prompt}")

        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",  # DALL-E 3는 1024x1024, 1792x1024, 1024x1792 크기를 지원합니다.
            quality="standard",  # standard 또는 hd
            n=1, # 생성할 이미지 개수
        )
        
        # 생성된 이미지의 URL을 반환합니다.
        image_url = response.data[0].url
        print(f"✅ 이미지 생성 성공! URL: {image_url}")
        return image_url

    except openai.APIError as e:
        print(f"OpenAI API 오류 발생: {e}")
        return None

# --- 실행 ---
if __name__ == "__main__":
    # 정책을 우회하기 위해 실존 인물 이름 대신 외형적 특징을 묘사
    safer_prompt = "An ultra-realistic RAW photo of an elderly woman sitting on a park bench. Late afternoon. Shot on a Sony α7 IV with an 85mm f/1.4 lens. Golden hour natural lighting, 8K resolution, extremely fine details, showing the texture of her skin, wrinkles, and individual strands of hair"
    
    generate_image_with_dalle3(safer_prompt)

🎨 DALL-E 3 이미지 생성을 시작합니다...
   프롬프트: An ultra-realistic RAW photo of an elderly woman sitting on a park bench. Late afternoon. Shot on a Sony α7 IV with an 85mm f/1.4 lens. Golden hour natural lighting, 8K resolution, extremely fine details, showing the texture of her skin, wrinkles, and individual strands of hair


AttributeError: 'str' object has no attribute 'images'

In [42]:
import os
import requests
import base64

def generate_hyper_realistic_image(prompt: str):
    """
    Stability AI API를 사용하여 극사실적인 이미지를 생성합니다.
    """
    api_host = 'https://api.stability.ai'
    api_key = os.getenv("STABILITY_API_KEY")
    if not api_key:
        raise ValueError("STABILITY_API_KEY 환경 변수를 설정해주세요.")

    engine_id = "stable-diffusion-xl-1024-v1-0"

    response = requests.post(
        f"{api_host}/v1/generation/{engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {api_key}"
        },
        json={
            "text_prompts": [
                {
                    "text": prompt
                }
            ],
            "cfg_scale": 7,
            "height": 1024,
            "width": 1024,
            "samples": 1,
            "steps": 40, # 퀄리티를 위해 스텝 수를 30~50으로 설정
            "style_preset": "photographic"  # ✨ 여기가 핵심! ✨
        },
    )

    if response.status_code != 200:
        raise Exception("API Error: " + str(response.json()))

    data = response.json()

    # 결과 이미지를 파일로 저장
    for i, image in enumerate(data["artifacts"]):
        with open(f"realistic_output_{i}.png", "wb") as f:
            f.write(base64.b64decode(image["base64"]))
    print("✅ 극사실적인 이미지가 'realistic_output_0.png'로 저장되었습니다.")


# --- 실행 ---
if __name__ == "__main__":
    # 프롬프트 엔지니어링 기법 적용
    final_prompt = "An ultra-realistic RAW photo of a quiet, old-fashioned library at night. A single leather armchair sits under a warm reading lamp. Shot on a Leica M11 with a 35mm f/1.4 lens, 8K, extremely fine details."
    
    generate_hyper_realistic_image(final_prompt)

✅ 극사실적인 이미지가 'realistic_output_0.png'로 저장되었습니다.


In [5]:
import os
import requests
import base64
import json
from dotenv import load_dotenv
import openai # openai 라이브러리 사용

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

def translate_with_gpt(text_to_translate: str) -> str:
    """
    OpenAI의 GPT 모델을 사용하여 텍스트를 영어로 번역합니다.
    """
    try:
        client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        if not client.api_key:
            raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

        response = client.chat.completions.create(
            model="gpt-4o",  # 최신 모델인 gpt-4o 사용 권장
            messages=[
                {"role": "system", "content": "You are a professional translator who translates Korean into natural, fluent English."},
                {"role": "user", "content": text_to_translate}
            ],
            temperature=0.3
        )
        
        translated_text = response.choices[0].message.content
        return translated_text if translated_text else ""

    except Exception as e:
        raise Exception(f"OpenAI API Error: {e}")


def generate_final_image(english_prompt: str):
    """
    사용자 요청을 분석하여, 기본 사진 스타일 또는 특정 아트 스타일로 이미지를 생성합니다.
    (이 함수는 이전과 동일합니다.)
    """
    # ... (이전과 동일한 '스마트' 이미지 생성 로직) ...
    style_keywords = [
        'style', 'anime', 'cartoon', 'painting', 'drawing', 'sketch',
        'gogh', 'picasso', 'dali', 'monet', 'watercolor'
    ]
    has_style_request = any(keyword in english_prompt.lower() for keyword in style_keywords)
    
    final_prompt = ""
    style_preset = "photographic"

    if has_style_request:
        print("🎨 스타일 요청을 감지했습니다.")
        final_prompt = english_prompt
        if 'anime' in english_prompt.lower(): style_preset = 'anime'
        elif 'comic' in english_prompt.lower(): style_preset = 'comic-book'
        else: style_preset = 'digital-art'
    else:
        print("📸 기본 '사진' 스타일로 이미지를 생성합니다.")
        photorealistic_prefix = "An ultra-realistic RAW photo of "
        photorealistic_suffix = ". Shot on a Sony α7 IV with an 85mm f/1.4 lens, 8K, cinematic lighting."
        final_prompt = photorealistic_prefix + english_prompt + photorealistic_suffix

    print(f"   - 최종 영어 프롬프트: {final_prompt}")
    print(f"   - 스타일 프리셋: {style_preset}")
    
    api_host = 'https://api.stability.ai'
    api_key = os.getenv("STABILITY_API_KEY")
    engine_id = "stable-diffusion-xl-1024-v1-0"

    if not api_key:
        raise ValueError("Stability AI API 키가 .env 파일에 설정되지 않았습니다.")
        
    print("   - 이미지 생성 요청 중...")
    response = requests.post(
        f"{api_host}/v1/generation/{engine_id}/text-to-image",
        headers={"Content-Type": "application/json", "Accept": "application/json", "Authorization": f"Bearer {api_key}"},
        json={
            "text_prompts": [{"text": final_prompt}], "cfg_scale": 7, "height": 1024,
            "width": 1024, "samples": 1, "steps": 30, "style_preset": style_preset
        },
    )

    if response.status_code != 200:
        raise Exception(f"Stability AI API Error: {response.status_code} - {response.text}")

    data = response.json()
    for i, image in enumerate(data["artifacts"]):
        output_path = f"output_image_{i}.png"
        with open(output_path, "wb") as f:
            f.write(base64.b64decode(image["base64"]))
        print(f"\n✅ 이미지가 '{output_path}' 경로에 성공적으로 저장되었습니다.")


# --- 메인 실행 부분 ---
if __name__ == "__main__":
    try:
        user_input = input("🎨 생성하고 싶은 이미지에 대해 한글 또는 영어로 자유롭게 설명해주세요: ")

        # ✨✨ GPT API로 번역하는 것으로 변경 ✨✨
        print("\n🔄 입력된 내용을 영어로 번역합니다 (with GPT)...")
        english_prompt = translate_with_gpt(user_input)
        print(f"   - 번역 결과: {english_prompt}")

        # 번역된 프롬프트로 이미지 생성
        generate_final_image(english_prompt)

    except Exception as e:
        print(f"\n--- 오류 발생 ---")
        print(e)
        print("-----------------")


🔄 입력된 내용을 영어로 번역합니다 (with GPT)...
   - 번역 결과: On the table, there is a plate with delicious-looking fresh pork belly.
📸 기본 '사진' 스타일로 이미지를 생성합니다.
   - 최종 영어 프롬프트: An ultra-realistic RAW photo of On the table, there is a plate with delicious-looking fresh pork belly.. Shot on a Sony α7 IV with an 85mm f/1.4 lens, 8K, cinematic lighting.
   - 스타일 프리셋: photographic
   - 이미지 생성 요청 중...

✅ 이미지가 'output_image_0.png' 경로에 성공적으로 저장되었습니다.
